In [5]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [6]:
import serial 
import time 
import schedule 
import gspread
from oauth2client.service_account import ServiceAccountCredentials 
from datetime import datetime

# Initialize the serial connection (adjust 'COM12' to your port)
arduino = serial.Serial('COM12', 9600, timeout=1)
time.sleep(2)  # Wait for the connection to stabilize

def push_data_to_cloud(room_occupancy, people, temp, humidity, time_occupied):
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
    ]
    creds_sample = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope) 
    client = gspread.authorize(creds_sample)
    sheet = client.open("CIA1comp2").sheet1

    # Check if the sheet is empty
    if not sheet.get_all_values():  # If the sheet is empty
        # Add column names
        column_names = ["Room Occupancy", "People", "Temp", "Humidity", "Time Occupied"]
        sheet.append_row(column_names)
        print('Column names added to the sheet.')

    # Prepare data to append
    data_to_append = [room_occupancy, people, temp, humidity, time_occupied] 
    sheet.append_row(data_to_append)
    print('Data pushed to cloud')

def main_func():
    try:
        # Read a line from the Arduino
        arduino_data = arduino.readline()  # Read the line from Arduino
        decoded_value = arduino_data.decode("utf-8").strip()  # Decode and strip whitespace

        # Split the decoded string into key-value pairs
        key_value_pairs = decoded_value.split(',')

        # Initialize variables
        room_occupancy = False
        people = temp = humidity = time_occupied = None

        # Process each key:value pair
        for pair in key_value_pairs:
            key, value = pair.split(':')  # Split by ':'
            key = key.strip()  # Remove any leading/trailing whitespace
            value = value.strip()  # Remove any leading/trailing whitespace

            # Assign values to the corresponding variables
            if key == "Room Occupancy":
                room_occupancy = int(value) > 0
            elif key == "People":
                people = int(value)
            elif key == "Temp":
                temp = float(value)
            elif key == "Humidity":
                humidity = float(value)
            elif key == "Time Occupied":
                time_occupied = int(value)

        # Print the values
        print(f'Room Occupancy: {room_occupancy}, People: {people}, Temp: {temp}, Humidity: {humidity}, Time Occupied: {time_occupied}')
        push_data_to_cloud(room_occupancy, people, temp, humidity, time_occupied)

    except ValueError:
        print("Error: Could not convert data to the expected format.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Main Code
print('Program started')

# Schedule the main function to run every 2 seconds
schedule.every(2).seconds.do(main_func) 

try:
    while True:
        schedule.run_pending()
        time.sleep(1)
except KeyboardInterrupt:
    print("Program terminated.")
finally:
    if arduino.is_open:
        arduino.close()  # Close the serial port when done
        print('Connection closed')

Program started
Error: Could not convert data to the expected format.
Room Occupancy: False, People: 1, Temp: 28.0, Humidity: 72.0, Time Occupied: 0
Data pushed to cloud
Room Occupancy: False, People: 2, Temp: 28.0, Humidity: 72.0, Time Occupied: 1
Data pushed to cloud
Room Occupancy: False, People: 3, Temp: 28.0, Humidity: 72.0, Time Occupied: 2
Data pushed to cloud
Room Occupancy: False, People: 4, Temp: 28.0, Humidity: 72.0, Time Occupied: 3
Data pushed to cloud
Room Occupancy: False, People: 5, Temp: 28.0, Humidity: 72.0, Time Occupied: 4
Data pushed to cloud
Room Occupancy: False, People: 6, Temp: 28.0, Humidity: 72.0, Time Occupied: 5
Data pushed to cloud
Room Occupancy: False, People: 7, Temp: 28.0, Humidity: 72.0, Time Occupied: 6
Data pushed to cloud
Room Occupancy: False, People: 8, Temp: 28.0, Humidity: 72.0, Time Occupied: 7
Data pushed to cloud
Room Occupancy: False, People: 9, Temp: 28.0, Humidity: 72.0, Time Occupied: 8
Data pushed to cloud
Room Occupancy: False, People: